In [80]:
# | export

import foodcosts.core as core
import matplotlib.pyplot as plt
import pandas as pd
import math
import smtplib
from datetime import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from IPython.display import display_markdown
import markdown


In [81]:
# | export

def get_days_ago():
    return 0


def get_today():
    return core.get_previous_n_day(get_days_ago())


def get_tomorrow():
    return core.get_previous_n_day(-1 + get_days_ago())


def get_weekday(date):
    date_object = datetime.strptime(date, '%Y-%m-%d')
    return date_object.strftime('%A')

def get_last_four_weekday_dates():
    last_four_weekday_dates = []
    for i in range(7, 29, 7):
        last_four_weekday_dates.append(
            core.get_previous_n_day(i+get_days_ago()))
    return last_four_weekday_dates


In [82]:
# | export

def send_mail(to='niloy.shown+a1@gmail.com', subject='', body=''):
    msg = MIMEMultipart()
    msg['From'] = 'niloy.shown@gmail.com'
    msg['To'] = to

    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    username = 'niloy.shown@gmail.com'
    password = 'xmkjodlqbzmkhmhb'

    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(username, password)

    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'html'))

    server.sendmail(msg['From'], msg['To'], msg.as_string())
    server.quit()


In [83]:
# | export

def get_email_subject(date):
    weekday = get_weekday(date)
    return f"""{date}'s - {weekday} -report"""


def get_daily_report_top_md(date, day, total_sales, shops_total_sales, re_restaurants_total_sales, events_total_sales):
    total_sales = int(total_sales)
    shops_total_sales = int(shops_total_sales)
    re_restaurants_total_sales = int(re_restaurants_total_sales)
    events_total_sales = int(events_total_sales)

    return f"""
# {date}'s - {day} -report

  

Today sales were {total_sales} EUR across the business:

  

- {shops_total_sales} EUR sold in stores

- {re_restaurants_total_sales} EUR sold in corporate restaurants

- {events_total_sales} EUR sold in events

        """


def get_store_sales_title_md():
    return f"""
## Store sales

### Stores sales per shop


        """


def get_sales_details_md(shop_name, total_sale, weekday, avg_sales, change_pct, change):
    total_sale = int(total_sale)
    avg_sales = int(avg_sales)
    change_pct = round(change_pct, 1)

    return f"""
- {shop_name} sold {total_sale} EUR. Average of last 4 {weekday}’s was {avg_sales} EUR. That’s {change_pct}% {change} vs. average of last 4 {weekday}'s

        """


def get_product_stock_out_title_md(outlet_type):
    return f"""
### Products that were likely in stock out

  

The products that were sold in the each days of last 7 days for a given {outlet_type} but didn’t sell today:

 
        """


def get_stock_out_product_details_md(product_name, outlet):
    return f"""
- {product_name} in {outlet}

        """


def get_corporate_restaurant_sales_title_md():
    return f"""
## Corporate restaurant sales

  

### Sales per corporate restaurant

  
  """


def get_corporate_restaurant_sales_md(name, total_sale, weekday, avg_sales, change_pct, change):
    total_sale = int(total_sale)
    avg_sales = int(avg_sales)
    change_pct = round(change_pct, 1)

    return f"""
- {name} sold {total_sale} EUR. Average of last 4 {weekday}’s was {avg_sales} EUR. That’s {change_pct}% {change} vs. average of last 4 {weekday}'s

        """


def get_no_sales_in_corporate_restaurant_md():

    return f"""
There is no sales in the corporate restaurant today

        """


In [84]:
# | export

def get_sales_details_html(sales_details_md, change):
    sales_details_html = markdown.markdown(sales_details_md)
    if (change == "increase"):
        return sales_details_html[:9] + f"""<span style="background-color:#00FF00">""" + sales_details_html[9:-11] + f"""</span>""" + sales_details_html[-11:]
    elif (change == "decrease"):
        return sales_details_html[:9] + f"""<span style="background-color:#FFA07A">""" + sales_details_html[9:-11] + f"""</span>""" + sales_details_html[-11:]
    else:
        return sales_details_html


In [85]:
# | export
def get_corporate_restaurant_names():
    return [name for (i, name) in core.make_sql_query(
    "select * from v_resto_entreprise")]

def get_all_shop_sales_as_pd(dates):
    dates_str = ", ".join("'" + str(val) + "'" for val in dates)

    query = f"""
        SELECT
            v_shop.id as id,
            v_shop.name as name,
            day,
            total
        FROM v_shop_sales_statistics
        LEFT JOIN
            v_shop on v_shop.id = v_shop_sales_statistics.shop_id
        WHERE
            v_shop.name IS NOT NULL AND
        day IN ({dates_str});
        """

    return core.get_query_res_as_pd(query)


def get_all_corporate_restaurant_sales_as_pd(dates):
    dates_str = ", ".join("'" + str(val) + "'" for val in dates)

    query = f"""
        SELECT 
            v_resto_entreprise.id as id,
            v_resto_entreprise.name as name,
            day,
            total
        FROM v_shop_sales_statistics
        LEFT JOIN 
            v_resto_entreprise on v_resto_entreprise.id = v_shop_sales_statistics.shop_id 
        WHERE 
        v_resto_entreprise.name IS NOT NULL AND     
        day IN ({dates_str});
        """

    return core.get_query_res_as_pd(query)


def get_all_event_sales_as_pd(start_date, end_date):
    query = f"""
        SELECT SUM(amount_total) as total
        FROM sale_order 
        WHERE 
        state = 'sale' AND date_order BETWEEN '{start_date}' and '{end_date}';
        """

    return core.get_query_res_as_pd(query)


In [87]:
# | export

def get_products_sold_for_shops_as_pd(start_date, end_date):
    query = f"""
        SELECT v_shop.name AS shop_name, 
               DATE(v_pos_order.created_at) AS day, 
               v_product.name AS product_name, 
               SUM(v_pos_order_line.quantity) AS units_sold, 
               SUM(v_pos_order_line.quantity * v_product.price) AS total_sales
        FROM v_pos_order_line
        LEFT JOIN v_product ON v_product.id = v_pos_order_line.product_id
        LEFT JOIN v_pos_order ON v_pos_order.id = v_pos_order_line.order_id
        LEFT JOIN v_pos_session ON v_pos_session.id = v_pos_order.session_id
        LEFT JOIN v_pos ON v_pos.id = v_pos_session.pos_id
        LEFT JOIN v_shop ON v_shop.id = v_pos.shop_id
        WHERE v_pos_order.created_at BETWEEN '{start_date}' AND '{end_date}'
        GROUP BY v_shop.id, v_shop.name, day, v_product.id, v_product.name
        ORDER BY v_shop.name, day ASC;
        """

    return core.get_query_res_as_pd(query)


def get_products_sold_for_re_as_pd(start_date, end_date):
    query = f"""
        SELECT v_resto_entreprise.name AS shop_name, 
               DATE(v_pos_order.created_at) AS day, 
               v_product.name AS product_name, 
               SUM(v_pos_order_line.quantity) AS units_sold, 
               SUM(v_pos_order_line.quantity * v_product.price) AS total_sales
        FROM v_pos_order_line
        LEFT JOIN v_product ON v_product.id = v_pos_order_line.product_id
        LEFT JOIN v_pos_order ON v_pos_order.id = v_pos_order_line.order_id
        LEFT JOIN v_pos_session ON v_pos_session.id = v_pos_order.session_id
        LEFT JOIN v_pos ON v_pos.id = v_pos_session.pos_id
        LEFT JOIN v_resto_entreprise ON v_resto_entreprise.id = v_pos.shop_id
        WHERE v_pos_order.created_at BETWEEN '{start_date}' AND '{end_date}'
        GROUP BY v_resto_entreprise.id, v_resto_entreprise.name, day, v_product.id, v_product.name
        ORDER BY v_resto_entreprise.name, day ASC;
    """
    return core.get_query_res_as_pd(query)


In [88]:
# | export

def get_sales_details_html(sales_details_md, change):
    sales_details_html = markdown.markdown(sales_details_md)
    if (change == "increase"):
        return sales_details_html[:9] + f"""<span style="background-color:#00FF00">""" + sales_details_html[9:-11] + f"""</span>""" + sales_details_html[-11:]
    elif (change == "decrease"):
        return sales_details_html[:9] + f"""<span style="background-color:#FFA07A">""" + sales_details_html[9:-11] + f"""</span>""" + sales_details_html[-11:]
    else:
        return sales_details_html


In [89]:
# | export

def compare_value_in_percentage(x, y):
    try:
        return round((x*100/(y)), 2)
    except ZeroDivisionError:
        return float('NaN')


def get_change_percentage(x, y):
    change_percentage = compare_value_in_percentage(x, y)
    change = "decrease" if change_percentage < 0 else "increase"
    abs_change_percentage = abs(change_percentage)

    return change, abs_change_percentage


In [96]:
# | export

def get_daily_report_top():
    today = get_today()
    tomorrow = get_tomorrow()

    all_shop_sales = get_all_shop_sales_as_pd([today])
    all_corporate_restaurant_sales = get_all_corporate_restaurant_sales_as_pd([
        today])
    all_event_sales = get_all_event_sales_as_pd(today, tomorrow)

    weekday = get_weekday(today)

    total_sales_at_shops = all_shop_sales['total'].values.sum()
    total_sales_at_corporate_restaurants = all_corporate_restaurant_sales['total'].values.sum(
    )
    total_sales_at_events = all_event_sales['total'].values.sum()

    print(total_sales_at_shops, total_sales_at_corporate_restaurants, total_sales_at_events)
    total_sales = total_sales_at_shops + \
        total_sales_at_corporate_restaurants + total_sales_at_events

    daily_report_top_md = get_daily_report_top_md(
        today,
        weekday,
        total_sales,
        total_sales_at_shops,
        total_sales_at_corporate_restaurants,
        total_sales_at_events)

    daily_report_top_html = markdown.markdown(daily_report_top_md)

    return daily_report_top_md, daily_report_top_html


In [91]:
# | export

def get_store_sales_title():
    store_sales_title_md = get_store_sales_title_md()
    store_sales_title_html = markdown.markdown(store_sales_title_md)

    return store_sales_title_md, store_sales_title_html


def get_sales_per_shop():
    today = get_today()
    last_four_weekday_dates = get_last_four_weekday_dates()

    all_shop_sales_for_last_four_weekdays = get_all_shop_sales_as_pd(
        last_four_weekday_dates)

    all_shop_sales = get_all_shop_sales_as_pd([today])
    shop_list = core.get_shop_names()

    sales_per_shop_md = ""
    sales_per_shop_html = ""
    for shop in shop_list:
        sale_by_shop = all_shop_sales[all_shop_sales['name']
                                      == shop]['total'].values.sum()
        sale_by_shop_in_last_four_weeks_at_this_weekday = all_shop_sales_for_last_four_weekdays[
            all_shop_sales_for_last_four_weekdays['name'] == shop]['total'].values.sum()
        avg_sale_at_this_weekday_in_last_four_week = round(
            (sale_by_shop_in_last_four_weeks_at_this_weekday / 4), 2)

        difference_in_sale = sale_by_shop - avg_sale_at_this_weekday_in_last_four_week

        change, change_percentage = get_change_percentage(
            difference_in_sale, avg_sale_at_this_weekday_in_last_four_week)

        shop_sales_md = get_sales_details_md(shop, sale_by_shop, get_weekday(
            today), avg_sale_at_this_weekday_in_last_four_week, change_percentage, change)

        sales_per_shop_md += shop_sales_md
        sales_per_shop_html += get_sales_details_html(shop_sales_md, change)

    return sales_per_shop_md, sales_per_shop_html


In [63]:
# | export

def get_shop_product_stock_out_title():
    shop_product_stock_out_title_md = get_product_stock_out_title_md("shop")
    shop_product_stock_out_title_html = markdown.markdown(
        shop_product_stock_out_title_md)

    return shop_product_stock_out_title_md, shop_product_stock_out_title_html


def get_shop_product_likely_to_stock_out():
    today = get_today()
    start_date = core.get_previous_n_day(7 + get_days_ago())
    end_date = core.get_previous_n_day(-1 + get_days_ago())

    shop_list = core.get_shop_names()

    product_sold_for_shops = {}
    products_sold_for_shops_pd = get_products_sold_for_shops_as_pd(
        start_date, end_date)

    for shop in shop_list:
        product_sold_for_shops[shop] = products_sold_for_shops_pd[products_sold_for_shops_pd['shop_name'] == shop]

    all_stock_out_product_md = ""
    all_stock_out_product_html = ""

    for shop in shop_list:
        products_sold_for_shops = product_sold_for_shops[shop]
        sales_frequency = products_sold_for_shops['product_name'].value_counts(
        )

        for product, count in sales_frequency.items():
            if (count == 7):
                sales_today = products_sold_for_shops.loc[(products_sold_for_shops['product_name'] == product) & (
                    pd.to_datetime(products_sold_for_shops['day']) == pd.to_datetime(today))]
                if (len(sales_today) == 0):
                    stock_out_product_details = get_stock_out_product_details_md(
                        product, shop)
                    all_stock_out_product_md += stock_out_product_details
                    all_stock_out_product_html += markdown.markdown(
                        stock_out_product_details)

    return all_stock_out_product_md, all_stock_out_product_html


In [92]:
# | export

def get_re_sales_title():
    re_sales_title_md = get_corporate_restaurant_sales_title_md()
    re_sales_title_html = markdown.markdown(re_sales_title_md)

    return re_sales_title_md, re_sales_title_html


def get_sales_per_re():
    today = get_today()
    last_four_weekday_dates = get_last_four_weekday_dates()

    all_re_sales_for_last_four_weekdays = get_all_corporate_restaurant_sales_as_pd(
        last_four_weekday_dates)

    all_re_sales = get_all_corporate_restaurant_sales_as_pd([today])
    shop_list = get_corporate_restaurant_names()

    all_shop_sales_md = ""
    all_shop_sales_html = ""
    for shop in shop_list:
        sale_by_shop = all_shop_sales[all_shop_sales['name']
                                      == shop]['total'].values.sum()
        sale_by_shop_in_last_four_weeks_at_this_weekday = all_shop_sales_for_last_four_weekdays[
            all_shop_sales_for_last_four_weekdays['name'] == shop]['total'].values.sum()
        avg_sale_at_this_weekday_in_last_four_week = round(
            (sale_by_shop_in_last_four_weeks_at_this_weekday / 4), 2)

        difference_in_sale = sale_by_shop - avg_sale_at_this_weekday_in_last_four_week

        change, change_percentage = get_change_percentage(
            difference_in_sale, avg_sale_at_this_weekday_in_last_four_week)

        shop_sales_md = get_shop_sales_md(shop, sale_by_shop, get_weekday(
            today), avg_sale_at_this_weekday_in_last_four_week, change_percentage, change)

        all_shop_sales_md += shop_sales_md
        all_shop_sales_html += get_sales_details_html(shop_sales_md, change)

    return all_shop_sales_md, all_shop_sales_html


In [94]:
# | export

def get_daily_report():
    daily_report_md = ""
    daily_report_html = ""

    daily_report_top_md, daily_report_top_html = get_daily_report_top()
    daily_report_md += daily_report_top_md
    daily_report_html += daily_report_top_html

    store_sales_title_md, store_sales_title_html = get_store_sales_title()
    daily_report_md += store_sales_title_md
    daily_report_html += store_sales_title_html

    sales_per_shop_md, sales_per_shop_html = get_sales_per_shop()
    daily_report_md += sales_per_shop_md
    daily_report_html += sales_per_shop_html

    # shop_product_stock_out_title_md, shop_product_stock_out_title_html = get_shop_product_stock_out_title()
    # daily_report_md += shop_product_stock_out_title_md
    # daily_report_html += shop_product_stock_out_title_html

    # all_stock_out_product_md, all_stock_out_product_html = get_shop_product_likely_to_stock_out()
    # daily_report_md += all_stock_out_product_md
    # daily_report_html += all_stock_out_product_html

    return daily_report_md, daily_report_html


In [97]:
# | export

daily_report_md, daily_report_html = get_daily_report()
display_markdown(daily_report_md, raw=True)

# send_mail(to='yann@fthek.be', subject=get_email_subject(get_today()),
#           body=daily_report_html)
# send_mail(to='niloy.shown+a1@gmail.com',
#           subject=get_email_subject(get_today()), body=daily_report_html)


config.ini file available... using that
config.ini file available... using that
config.ini file available... using that
0 0 None


TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'